In [1]:
import pandas as pd
import sqlite3

In [2]:
db = sqlite3.connect('db.sqlite3')

In [3]:
cu = db.cursor()

In [4]:
# Recuperation des données des differentes tables de notre BD
patients = pd.read_sql("""
                       SELECT id, sexe, age, ethnie, profession, quartier, religion, statut_matrimonial 
                       FROM Patients_patients""",
                       db)
antecedentpat = pd.read_sql("""
                        SELECT id, patient_id, medicaux, alcool_imm, allergies_imm, autres_gyn, autres_imm, chirurgie  
                        FROM Patients_antecedentpatients""", 
                        db)
observation = pd.read_sql("""
                        SELECT id, patient_id FROM Consultation_observation_medicale""", 
                        db)
consultation = pd.read_sql("""
                        SELECT id, patient_id, motif_consultation, histoire_maladie, clinique, enquete_systeme, exam_physique,
                           conduite_a_tenir, diagnostique
                        FROM Consultation_consultation""", 
                        db)
parametre_entree = pd.read_sql('SELECT id, parametre, valeur, observation_id FROM Hospitalisation_parametre_entree', db)

In [5]:
parametre = pd.DataFrame()
parametre['patient_id'] = observation['patient_id']
for i,row in parametre_entree.iterrows():
    for j, obs in observation.iterrows():
        if row['observation_id'] == obs['id']:
            parametre[row['parametre']] = None
            parametre[row['parametre']][j] = row['valeur']

for i, row in parametre_entree.iterrows():
    for j, obs in observation.iterrows():
        if row['observation_id'] == obs['id']:
            parametre[row['parametre']][j] = row['valeur']
parametre

C:\Users\personnel\AppData\Local\Temp\ipykernel_2236\2018946192.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parametre[row['parametre']][j] = row['valeur']
C:\Users\personnel\AppData\Local\Temp\ipykernel_2236\2018946192.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parametre[row['parametre']][j] = row['valeur']
C:\Users\personnel\AppData\Local\Temp\ipykernel_2236\2018946192.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,patient_id,IMC,IT,Poids,Saturation en O2,Taille,Température,Tension Arterielle (BD),Tension Arterielle (BG),Fréquence cardiaque (FC),Fréquence respiratoire (FR),Conscience (SCG),Glycemie
0,40,22,65,54,99,157,36.9,126,115,None,None,None,None
1,41,None,None,None,None,None,None,None,None,None,None,None,None
2,42,29.8,103,85,99,169,36.5,114,110,None,None,None,None
3,43,26.2,None,88,98,183,36.7,93,92,None,None,None,None
4,44,27.7,None,71,100,161,None,112,103,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,233,27.3,82,71,100,160,37,108,116,None,None,None,None
193,234,21,105,63,99,173,37.3,92,98,None,None,None,None
194,235,43.8,99,106,96,155,37.6,142,117,None,None,None,None
195,236,27.2,74,80,99,171,37,93,89,None,None,None,None


In [6]:
parametre_consultation = parametre.merge(consultation, right_on='patient_id', left_on='patient_id', how='inner')
parametre_consultation

,patient_id,IMC,IT,Poids,Saturation en O2,Taille,Température,Tension Arterielle (BD),Tension Arterielle (BG),Fréquence cardiaque (FC),...,Conscience (SCG),Glycemie,id,motif_consultation,histoire_maladie,clinique,enquete_systeme,exam_physique,conduite_a_tenir,diagnostique
0,40,22,65,54,99,157,36.9,126,115,None,...,None,None,19,"visite de suivi: cephalés, cervicalgie, vertig...",Le debut de symptomatologie remonterai à envir...,None,,"conscient, cooperant, bien orienté dans le tem...","Continuer T3, Glycemie, surveillance TA3",None
1,42,29.8,103,85,99,169,36.5,114,110,None,...,None,None,20,Adressé par l'endocrinologue pour PEC nutritio...,,None,"Prend 3 repas par jour, consomme beaucoup de f...",,Suivre un regime alimentaire hypocalorique de ...,Obésité probable
2,43,26.2,None,88,98,183,36.7,93,92,None,...,None,None,21,,Evolution des symptomes depuis 2jrs,None,,Bon etat general,None,gastro-entérite
3,44,27.7,None,71,100,161,None,112,103,None,...,None,None,22,Tumefaction sous ambilicale. indolore reductib...,,None,,,RDV avec bilan,"Hernie epigastrique, hernie de la ligne blanch"
4,46,27.9,66,65,100,153,None,74,69,None,...,None,None,23,"Asthénie, vertige, cephale",,"Asthénie, vertige, cephale",,conscience SCG 15/15.. reste sp,"Efferalgan 1cp *3, Hemafer 1cp/jr","signe syptomatique de la grossesse, paludisme"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,233,27.3,82,71,100,160,37,108,116,None,...,None,None,214,"courbature, polyarthralgie depuis 48h",plusieurs automedications,None,"fievre, courbature, polyarthralgie",BEG,"bilan: GE, uricemie, NFS",Paludisme simple sur RAA
196,234,21,105,63,99,173,37.3,92,98,None,...,None,None,215,"Fievre, asthenie, courbature, anorexie",,None,"Fievre, asthenie, courbature, anorexie","BEG, conjonctive coloree","bilan: GE, TDR, CRP",paludisme simple
197,235,43.8,99,106,96,155,37.6,142,117,None,...,None,None,216,"Courbature, vertige, cephale + presentation ep...",,None,"Courbature, vertige, cephale","BEG, conjonctive rosee, sclere anicterique, se...","bilan: GE, NFS, TDR",Paludisme simple sur gastrite aigue
198,236,27.2,74,80,99,171,37,93,89,None,...,None,None,217,Douleur peluvienne evoluant depuis 3 ans,le debut remonte à 3 ans par l'installation pr...,None,"gonalgie, douleur lombaire, douleur abdominale...","conjonctive coloree, cslere anicterique, cavit...","bilan: GE, NFS, ECBU, ionogramme complet","lombosciatalgie, paludisme simple"


In [7]:
antecedent_patients = patients.merge(antecedentpat, left_on='id', right_on='patient_id', how='left')
antecedent_patients

,id_x,sexe,age,ethnie,profession,quartier,religion,statut_matrimonial,id_y,patient_id,medicaux,alcool_imm,allergies_imm,autres_gyn,autres_imm,chirurgie
0,40,Feminin,65,None,Retraitée,BIBEMIRE (Garoua),Musulmane,Marié(e) Monogame,8.0,40.0,HTA depuis 10 ans,0.0,None,ménopause,None,None
1,41,Masculin,9,None,Eleve,BIBEMIRE (Garoua),Musulmane,Celibataire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42,Masculin,28,None,ElectroTechnicien,Tchad,Musulmane,Celibataire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43,Masculin,33,None,Aide commerçant,LAINDE daniel (Garoua),Musulmane,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44,Feminin,45,Peulh,Ménagère,TOUBORO (Garoua),Musulmane,Marié(e) Polygame,9.0,44.0,None,0.0,"Niaquine, Quinine",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,233,Feminin,46,BANA,Enseignante,Marouaré (Garoua),Chretienne,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,234,Feminin,49,Mafa,TECHNICIEN SUPERIEUR OPHTALMO,None,Chretienne,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,235,Feminin,58,LAKA,Ménagère,Mairie (Garoua),Chretienne,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,236,Feminin,45,Moudang,Ménagère,Lagdo (Garoua),Catholique,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
donnees_patients = parametre_consultation.merge(antecedent_patients, left_on='patient_id', right_on='id_x', how='left')
donnees_patients

,patient_id_x,IMC,IT,Poids,Saturation en O2,Taille,Température,Tension Arterielle (BD),Tension Arterielle (BG),Fréquence cardiaque (FC),...,religion,statut_matrimonial,id_y,patient_id_y,medicaux,alcool_imm,allergies_imm,autres_gyn,autres_imm,chirurgie
0,40,22,65,54,99,157,36.9,126,115,None,...,Musulmane,Marié(e) Monogame,8.0,40.0,HTA depuis 10 ans,0.0,None,ménopause,None,None
1,42,29.8,103,85,99,169,36.5,114,110,None,...,Musulmane,Celibataire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43,26.2,None,88,98,183,36.7,93,92,None,...,Musulmane,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,44,27.7,None,71,100,161,None,112,103,None,...,Musulmane,Marié(e) Polygame,9.0,44.0,None,0.0,"Niaquine, Quinine",None,None,None
4,46,27.9,66,65,100,153,None,74,69,None,...,Chretienne,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,233,27.3,82,71,100,160,37,108,116,None,...,Chretienne,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,234,21,105,63,99,173,37.3,92,98,None,...,Chretienne,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,235,43.8,99,106,96,155,37.6,142,117,None,...,Chretienne,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,236,27.2,74,80,99,171,37,93,89,None,...,Catholique,Marié(e) Monogame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
donnees_patients.columns

Index(['patient_id_x', 'IMC', 'IT', 'Poids', 'Saturation en O2', 'Taille',
       'Température', 'Tension Arterielle (BD)', 'Tension Arterielle (BG)',
       'Fréquence cardiaque (FC)', 'Fréquence respiratoire (FR)',
       'Conscience (SCG)', 'Glycemie', 'id', 'motif_consultation',
       'histoire_maladie', 'clinique', 'enquete_systeme', 'exam_physique',
       'conduite_a_tenir', 'diagnostique', 'id_x', 'sexe', 'age', 'ethnie',
       'profession', 'quartier', 'religion', 'statut_matrimonial', 'id_y',
       'patient_id_y', 'medicaux', 'alcool_imm', 'allergies_imm', 'autres_gyn',
       'autres_imm', 'chirurgie'],
      dtype='object')

In [10]:
supprime = ['id_y', 'patient_id_x', 'patient_id_y', 'id_x']
donnees_patients.drop(supprime, axis=1, inplace=True)

In [11]:
donnees_patients.to_csv('Patients_data.csv', index=False, encoding='utf8')